In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
import tensorflow as tf
import random
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SimpleRNN

2024-09-28 15:17:34.879242: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-28 15:17:34.879372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-28 15:17:35.030216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/amirkabirdam/holiday.csv
/kaggle/input/amirkabirdam/dam12.csv
/kaggle/input/climate-indeices/ONI.xlsx
/kaggle/input/climate-indeices/ENSO.xlsx
/kaggle/input/climate-indeices/AMO.xlsx
/kaggle/input/climate-indeices/NAO.xlsx
/kaggle/input/climate-indeices/SOI.xlsx
/kaggle/input/d/homaabedinzadeh/holidayname/holidayname.csv
/kaggle/input/d/homaabedinzadeh/futuredays/90days.csv


In [3]:
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

In [4]:
oni = pd.read_excel('../input/climate-indeices/ONI.xlsx')
enso = pd.read_excel('../input/climate-indeices/ENSO.xlsx')
#amo = pd.read_excel('../input/climate-indeices/AMO.xlsx')
nao = pd.read_excel('../input/climate-indeices/NAO.xlsx')
soi = pd.read_excel('../input/climate-indeices/SOI.xlsx')


In [5]:
oni=pd.melt(oni, id_vars=['Year'], var_name='month', value_name='value').sort_values(by=['Year','month'])
enso=pd.melt(enso, id_vars=['Year'], var_name='month', value_name='value').sort_values(by=['Year','month'])
#amo=pd.melt(amo, id_vars=['Year'], var_name='month', value_name='value').sort_values(by=['Year','month'])
nao=pd.melt(nao, id_vars=['Year'], var_name='month', value_name='value').sort_values(by=['Year','month'])
soi=pd.melt(soi, id_vars=['Year'], var_name='month', value_name='value').sort_values(by=['Year','month'])

In [6]:
climate=soi.merge( nao ,how='left', on= ['Year', 'month'],suffixes=('_s', '_n') ).merge(oni,how='left', on= ['Year', 'month']).merge(enso,how='left', on= ['Year', 'month'],suffixes=('_a', '_e'))#.merge(amo,how='left', on= ['Year', 'month'])
climate.head(2)

,Year,month,value_s,value_n,value_a,value_e
0,1979,01,0.2,-1.38,0.0,0.26
1,1979,02,0.2,-0.67,0.1,-0.25


In [7]:
climate=climate.rename(columns={'value_s':'soi' ,'value_n':'nao' , 'value_a':'amo', 'value_e':'enso', 'value':'oni'})
climate.dropna(inplace=True)

In [8]:
climate.head(2)

,Year,month,soi,nao,amo,enso
0,1979,01,0.2,-1.38,0.0,0.26
1,1979,02,0.2,-0.67,0.1,-0.25


In [9]:

climate['soi360']=climate['soi'].shift(12)

climate['nao30']=climate['nao'].shift(1)
climate['nao90']=climate['nao'].shift(3)
climate['nao360']=climate['nao'].shift(12)
climate['nao120']=climate['nao'].shift(4)
climate['nao150']=climate['nao'].shift(5)
climate['nao180']=climate['nao'].shift(6)
climate['nao210']=climate['nao'].shift(7)
climate['nao240']=climate['nao'].shift(8)
climate['nao270']=climate['nao'].shift(9)


climate['amo30']=climate['amo'].shift(1)
climate['amo90']=climate['amo'].shift(3)


climate['enso30']=climate['enso'].shift(1)
climate['enso90']=climate['enso'].shift(3)
climate['enso360']=climate['enso'].shift(12)

In [10]:
df = pd.read_csv('../input/amirkabirdam/dam12.csv')
df['Total_in'].head(2)

0    2.80800
1    0.28512
Name: Total_in, dtype: float64

In [11]:
df['humidity']=df[['NamNesbi6_5', 'NamNesbi12_5','NamNesbi18_5']].mean(axis=1)
df['daraje']= df[['MinDarajehararat_in','MaxDarajehararat_in']].mean(axis=1)

In [12]:
df['Total_in1']=df['Total_in'].shift(1)
df['Total_in2']=df['Total_in'].shift(2)
df['Total_in3']=df['Total_in'].shift(3)
df['Total_in4']=df['Total_in'].shift(4)
df['Total_in5']=df['Total_in'].shift(5)
df['Total_in6']=df['Total_in'].shift(6)
df['Total_in7']=df['Total_in'].shift(7)
df['Total_in30']=df['Total_in'].shift(30)
df['daraje30mean']=df['daraje'].shift(1).rolling(window=30).mean()

df['Barandegi_in1']=df['Barandegi_in'].shift(1)
df['Barandegi_in2']=df['Barandegi_in'].shift(2)
df['Barandegi_in30']=df['Barandegi_in'].shift(30)
df['Barandegi_in7']=df['Barandegi_in'].shift(7)

df['DebiSeilabm3c180']=df['DebiSeilabm3c'].shift(180)
df['DebiSeilabm3c120']=df['DebiSeilabm3c'].shift(120)
df['DebiSeilabm3c150']=df['DebiSeilabm3c'].shift(150)
df['DebiSeilabm3c270']=df['DebiSeilabm3c'].shift(2700)

In [13]:
df = df[df['DateRow']>'2015']

In [14]:
df.dropna(axis=0, inplace=True)

In [15]:
target = 'Total_in'
features = ['Total_in1','Total_in2','Total_in3','Total_in4','Total_in5','Total_in6','Total_in7',
            'Total_in30','daraje30mean', 'Barandegi_in2','Barandegi_in1','Barandegi_in7','Barandegi_in30'
            ,'Barandegi_in','TotalAbBarfVBaran_in','DebiSeilabm3c','daraje','VazeHava_in','SoratBad_in' ,'humidity'
           ,'DebiSeilabm3c150','DebiSeilabm3c120','DebiSeilabm3c180','DebiSeilabm3c270']

In [16]:
df['DateRow'] = pd.to_datetime(df['DateRow'])
df['Year'] = df['DateRow'].dt.year
df['month'] = df['DateRow'].dt.month
cal=['month','Year']
df = df.sort_values(by= 'DateRow')

df =pd.concat ([df[features], df[cal],df [target]], axis=1)


In [17]:
climate['month']=climate['month'].astype('int32')

In [18]:
df= climate.merge(df , how= 'inner' , on = ['Year', 'month'],suffixes=('_x', '_y'))

In [19]:
df.drop(columns=['Year','month'], axis=1 , inplace=True)

In [20]:
m=pd.DataFrame(df.corr()['Total_in'])


In [21]:
df=df [m[m['Total_in']>0.1].index]

In [22]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df.values.reshape(-1, 1))
#scaler.fit_transform(m.values.reshape(-1, 1))

In [23]:
def create_dataset(dataset, time_step=10):
    X, y = [], []
    for i in range(len(dataset)-time_step-forecast_horizon):
        X.append(dataset[i:(i+time_step), :])
        y.append(dataset[i+time_step+forecast_horizon, -1])
    return np.array(X), np.array(y)

time_step = 180
forecast_horizon = 30
#X, y = create_dataset_multistep(scaled_data, time_step, forecast_horizon)

In [24]:
X_rnn, y_rnn = create_dataset(scaled_data, time_step)

In [25]:

X_rnn = X_rnn.reshape(X_rnn.shape[0], X_rnn.shape[1], X_rnn.shape[2])


In [26]:
tscv = TimeSeriesSplit(n_splits = 7)
train_rmse = []
test_rmse = []
train_r2 = []
test_r2 = []
i=0
for train_index, test_index in tscv.split(X_rnn):
    i+=1
    X_train, X_test = X_rnn[train_index], X_rnn[test_index]
    Y_train, Y_test = y_rnn[train_index], y_rnn[test_index]
    if i>=3:
        model = Sequential()
        model.add(SimpleRNN(128, activation='relu', input_shape=(X_rnn.shape[1], X_rnn.shape[2])))
        model.add(Dropout(0.2))
        model.add(Dense(64))
        model.add(Dropout(0.2))
        model.add(Dense(64))
        model.add(Dense(1))  # خروجی یک روز بعد (پیش‌بینی)

        # کامپایل مدل
        model.compile(optimizer='adam', loss='mean_squared_error')
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        # آموزش مدل
        model.fit(X_rnn, y_rnn, epochs=10, batch_size=64,callbacks=[early_stopping],validation_data=(X_test, Y_test))
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)
        
        train_rmse .append (np.sqrt(mean_squared_error(Y_train, train_predict)))
        test_rmse.append ( np.sqrt(mean_squared_error(Y_test, test_predict)))
        train_r2 .append ( r2_score(Y_train, train_predict))
        test_r2 .append ( r2_score(Y_test, test_predict))
    
print(f"Train RMSE: {np.mean(train_rmse)}, Test RMSE: {np.mean(test_rmse)}")
print(f"Train R2: {np.mean(train_r2)}, Test R2: {np.mean(test_r2)}")
    

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 32s 48ms/step - loss: 7.6416e-04 - val_loss: 4.9106e-04
Epoch 2/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.7112e-04 - val_loss: 4.3922e-04
Epoch 3/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.5611e-04 - val_loss: 4.6944e-04
Epoch 4/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.4699e-04 - val_loss: 4.8316e-04
Epoch 5/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.5425e-04 - val_loss: 5.2607e-04
Epoch 6/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.4198e-04 - val_loss: 5.8717e-04
Epoch 7/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.4289e-04 - val_loss: 4.9227e-04
480/480 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


639/639 ━━━━━━━━━━━━━━━━━━━━ 32s 47ms/step - loss: 9.4223e-04 - val_loss: 3.8553e-04
Epoch 2/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.6948e-04 - val_loss: 4.2222e-04
Epoch 3/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.6821e-04 - val_loss: 4.0191e-04
Epoch 4/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.5377e-04 - val_loss: 4.5075e-04
Epoch 5/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.4638e-04 - val_loss: 5.1988e-04
Epoch 6/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.3675e-04 - val_loss: 5.0778e-04
640/640 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


639/639 ━━━━━━━━━━━━━━━━━━━━ 32s 47ms/step - loss: 0.0012 - val_loss: 8.1274e-04
Epoch 2/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.8046e-04 - val_loss: 7.5496e-04
Epoch 3/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.6362e-04 - val_loss: 7.9576e-04
Epoch 4/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.6038e-04 - val_loss: 7.5949e-04
Epoch 5/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.3897e-04 - val_loss: 8.4647e-04
Epoch 6/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.4257e-04 - val_loss: 8.2198e-04
Epoch 7/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.4795e-04 - val_loss: 8.4196e-04
799/799 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


639/639 ━━━━━━━━━━━━━━━━━━━━ 32s 47ms/step - loss: 0.0011 - val_loss: 7.2910e-04
Epoch 2/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.8091e-04 - val_loss: 7.0196e-04
Epoch 3/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.6516e-04 - val_loss: 7.0938e-04
Epoch 4/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.6552e-04 - val_loss: 7.3782e-04
Epoch 5/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.5824e-04 - val_loss: 7.1706e-04
Epoch 6/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.5309e-04 - val_loss: 7.2558e-04
Epoch 7/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 4.5683e-04 - val_loss: 8.2691e-04
959/959 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


639/639 ━━━━━━━━━━━━━━━━━━━━ 32s 47ms/step - loss: 8.3363e-04 - val_loss: 3.0287e-04
Epoch 2/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.6769e-04 - val_loss: 2.5565e-04
Epoch 3/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.6433e-04 - val_loss: 2.7219e-04
Epoch 4/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.5855e-04 - val_loss: 2.9352e-04
Epoch 5/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.4177e-04 - val_loss: 2.9705e-04
Epoch 6/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.4303e-04 - val_loss: 2.9857e-04
Epoch 7/10
639/639 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - loss: 4.5008e-04 - val_loss: 3.3457e-04
1119/1119 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Train RMSE: 0.019323280293920917, Test RMSE: 0.02211057955649614
Train R2: 0.8595684716718663, Test R2: 0.8262845743246048


In [27]:
def create_dataset(dataset, time_step=10 , forecast_horizon=1):
    X, y = [], []
    for i in range(len(dataset)-time_step-forecast_horizon):
        X.append(dataset[i:(i+time_step),: ])
        y.append(dataset[i+time_step+forecast_horizon, -1])
    return np.array(X), np.array(y)

time_step = 90
forecast_horizon =1
X, y = create_dataset(scaled_data, time_step, forecast_horizon)
print(X.shape)  
print(y.shape)

(41013, 90, 1)
(41013,)


In [28]:
tscv = TimeSeriesSplit(n_splits = 7)
train_rmse = []
test_rmse = []
train_r2 = []
test_r2 = []
i=0
for train_index, test_index in tscv.split(X):
    i+=1
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]
    if i>=3:
        model = Sequential()
        model.add(SimpleRNN(128, activation='relu', input_shape=(X.shape[1], X.shape[2])))
        model.add(Dropout(0.2))
        model.add(Dense(64))
        model.add(Dropout(0.2))
        model.add(Dense(64))
        model.add(Dense(1))  # خروجی یک روز بعد (پیش‌بینی)

        # کامپایل مدل
        model.compile(optimizer='adam', loss='mean_squared_error')
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
 
        model.fit(X, y, epochs=10, batch_size=64,callbacks=[early_stopping],validation_data=(X_test, Y_test))
    
        predicted_1_day = model.predict(X[-1].reshape(1, 90, 1)) 
        
        #train_predict = model.predict(X_train)
        #test_predict = model.predict(X_test)
        
        predicted_1_day_original = scaler.inverse_transform(predicted_1_day)
        predicted_90_days = []
        last_90_days = X[-1]
        for _ in range(90):
            pred = model.predict(last_90_days.reshape(1, 90, 1))
            predicted_90_days.append(pred[0][0])

            # به‌روزرسانی پنجره با اضافه کردن مقدار پیش‌بینی شده و حذف اولین مقدار
            last_90_days = np.append(last_90_days[1:], pred, axis=0)

            # مقیاس‌دهی معکوس برای کل 90 روز پیش‌بینی‌شده
            predicted_90_days_original = scaler.inverse_transform(np.array(predicted_90_days).reshape(-1, 1))
            
            
        

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


641/641 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - loss: 0.0024 - val_loss: 4.6472e-04
Epoch 2/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 6.1899e-04 - val_loss: 4.6786e-04
Epoch 3/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 5.8996e-04 - val_loss: 4.3441e-04
Epoch 4/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 5.8069e-04 - val_loss: 4.3802e-04
Epoch 5/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 5.5030e-04 - val_loss: 4.5789e-04
Epoch 6/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 5.5264e-04 - val_loss: 4.7825e-04
Epoch 7/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 5.7404e-04 - val_loss: 0.0028
Epoch 8/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - loss: 0.0028 - val_loss: 0.0028
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


641/641 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - loss: 0.0028 - val_loss: 0.0017
Epoch 2/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0013 - val_loss: 4.9665e-04
Epoch 3/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 6.2371e-04 - val_loss: 5.8336e-04
Epoch 4/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 6.7650e-04 - val_loss: 5.6217e-04
Epoch 5/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 6.2455e-04 - val_loss: 4.6791e-04
Epoch 6/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 5.7655e-04 - val_loss: 0.0027
Epoch 7/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0027 - val_loss: 0.0025
Epoch 8/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0025 - val_loss: 0.0017
Epoch 9/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0023 - val_loss: 0.0028
Epoch 10/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0026 - val_loss: 0.0019
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━

...

641/641 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - loss: 0.0028 - val_loss: 9.2975e-04
Epoch 2/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 6.8361e-04 - val_loss: 9.5566e-04
Epoch 3/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 6.2771e-04 - val_loss: 7.8496e-04
Epoch 4/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 7.8172e-04 - val_loss: 0.0026
Epoch 5/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 6/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0024 - val_loss: 0.0020
Epoch 7/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0022 - val_loss: 0.0028
Epoch 8/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0025 - val_loss: 0.0023
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


641/641 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - loss: 0.0029 - val_loss: 8.7660e-04
Epoch 2/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 7.0852e-04 - val_loss: 8.0568e-04
Epoch 3/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 5.9135e-04 - val_loss: 7.5197e-04
Epoch 4/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 0.0013 - val_loss: 0.0033
Epoch 5/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0026 - val_loss: 0.0032
Epoch 6/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0026 - val_loss: 0.0034
Epoch 7/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0028 - val_loss: 0.0033
Epoch 8/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0026 - val_loss: 0.0021
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


641/641 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - loss: 0.0024 - val_loss: 4.2255e-04
Epoch 2/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 6.6724e-04 - val_loss: 0.0026
Epoch 3/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 4/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.0024 - val_loss: 5.9183e-04
Epoch 5/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 7.8508e-04 - val_loss: 4.2179e-04
Epoch 6/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 6.7804e-04 - val_loss: 4.4043e-04
Epoch 7/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - loss: 6.2041e-04 - val_loss: 4.2384e-04
Epoch 8/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 6.0421e-04 - val_loss: 4.2892e-04
Epoch 9/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 5.6972e-04 - val_loss: 3.5224e-04
Epoch 10/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 5.6585e-04 - val_loss: 4.1511e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22m

In [29]:
predicted_90_days_original=pd.DataFrame (predicted_90_days_original)
predicted_90_days_original.head(2)

,0
0,0.017562
1,0.285495


In [30]:
features_days = pd.read_csv('../input/d/homaabedinzadeh/futuredays/90days.csv')
features_days.head(2)

,Total_in
0,0.23158
1,0.22716


In [31]:
features_days.shape

(90, 1)

In [32]:
print (r2_score(features_days, predicted_90_days_original))

-1.0444371928439655
